In [2]:
import pandas as pd
from IPython.display import display, Markdown
import os
import numpy as np
csv_folder = "../csvs/anova/"
## CASO NÃO ESTEJA ENCONTRANDO OS ARQUIVOS, SÓ PEGA E DESCOMENTA A LINHA DEBAIXO PRA ELE PEGAR E TROCAR DE DIRETÓRIO
# os.chdir(f"../{csv_folder}")

In [3]:
dataframe = pd.read_csv(csv_folder+"teste.csv")
print(dataframe)

    A   B   C   D   E   F
0  64  69  46  56  39  46
1  59  53  48  45  59  50
2  50  51  43  45  53  65
3  63  55  35  42  53  59


In [4]:
# NUMERO DE TRATAMENTOS E NUMERO DE REPETIÇÕES
r, k = dataframe.shape

## GRAUS DE LIBERDADE = GL = DF ## 
# Numero de Variaveis Independentes
# gl_trat = k - 1
df_between = k - 1
# gl_erro
df_within = (r*k) - k
print(f"(dfb) DF tratamentos(entre colunas): {round(df_between, 2)}")
print(f"(dfw) DF erros(dentro das colunas): {round(df_within, 2)}")

(dfb) DF tratamentos(entre colunas): 5
(dfw) DF erros(dentro das colunas): 18


In [5]:
# SUM OF SQUARE - SSQ
# Soma dos quadrados - SQ

mean_of_all_data = dataframe.to_numpy().mean()

# SSB ou SQTRAT
sum_of_square_between = 0 
# sum_of_square_between = sum([*map(lambda n: ((n - mean_of_all_data) ** 2) * r, dataframe.mean())])

for group in dataframe.items():
    column_mean = group[1].mean()
    for value in group[1]:
        sum_of_square_between += (column_mean - mean_of_all_data) ** 2  

print(f"(SSB = SQTrat) Soma de quadrados entre colunas: {sum_of_square_between}")

# SSW ou SQerro
sum_of_square_within = 0
# sum_of_square_within = sum([*map(lambda n: sum([*map(lambda m: (m - dataframe[n].mean())**2, dataframe[n])]), dataframe.columns)])

for value in dataframe.items():
    column_mean = value[1].mean()
    sum_of_column = sum([*map(lambda n: (n - column_mean) ** 2, value[1])])
    sum_of_square_within += sum_of_column

print(f"(SSW = SQErro) Soma de quadrados dentro de colunas: {sum_of_square_within}")

# SST ou SQTotal
sum_of_square_total = 0

for group in dataframe.iterrows():
    for value in group[1]:
        sum_of_square_total += (float(value) - mean_of_all_data) ** 2
print(f"(SST = SQTotal) Soma de quadrados Totais: {sum_of_square_total}")

(SSB = SQTrat) Soma de quadrados entre colunas: 760.0
(SSW = SQErro) Soma de quadrados dentro de colunas: 972.0
(SST = SQTotal) Soma de quadrados Totais: 1732.0


In [6]:
# MSw ou QMerro 
mean_square_within = sum_of_square_within / df_within
print(f"(MSw = QMerro) Media de quadrados entre as colunas {round(mean_square_within, 2)}")
# MSb ou QMtrat
mean_square_between = sum_of_square_between / df_between
print(f"(MSb = QMtrat) Media de quadrados entre colunas {round(mean_square_between, 2)}")
    

(MSw = QMerro) Media de quadrados entre as colunas 54.0
(MSb = QMtrat) Media de quadrados entre colunas 152.0


In [7]:
calculated_fischer = mean_square_between / mean_square_within
print(f"Fischer estatisticas: {calculated_fischer}")

variance_coefficient = np.sqrt(mean_square_within) / mean_of_all_data
variance_coefficient = variance_coefficient * 100
print(f"CV : {variance_coefficient} %")

Fischer estatisticas: 2.814814814814815
CV : 14.131671592979874 %


In [8]:
tabela_dict = {
"Fonte de Váriação" : ["Tratamento", "Erro", "Total", "Média Geral", "CV(%)"],
"GL": [df_between.__round__(2), df_within.__round__(2), df_between + df_within, "", ""],
"SQ": [sum_of_square_between.__round__(2), sum_of_square_within.__round__(2), sum_of_square_total.__round__(2),"", ""], 
"QM": [mean_square_between.__round__(2),mean_square_within.__round__(2),"","", ""], 
"F" : [calculated_fischer.__round__(2),"","", mean_of_all_data.__round__(2),variance_coefficient.__round__(2)]}

dataframe_tabela = pd.DataFrame(tabela_dict)

display(Markdown(dataframe_tabela.to_markdown(index=False)))

| Fonte de Váriação   | GL   | SQ     | QM    | F     |
|:--------------------|:-----|:-------|:------|:------|
| Tratamento          | 5    | 760.0  | 152.0 | 2.81  |
| Erro                | 18   | 972.0  | 54.0  |       |
| Total               | 23   | 1732.0 |       |       |
| Média Geral         |      |        |       | 52.0  |
| CV(%)               |      |        |       | 14.13 |

In [9]:
dataframe = pd.read_csv(f"{csv_folder}tabela_fischer.csv")

ftab = list(dataframe.iterrows())[df_within -1][1][f"{df_between}"]
print(f"Valor na Tabela de Fischer: {ftab}")
print(f"Valor calculado na tabela de variação: {calculated_fischer}")
if ftab < calculated_fischer:
    print("Existe Diferença Significativa")
else: 
    print("Não Existe Diferença Significativa")

Valor na Tabela de Fischer: 2.773
Valor calculado na tabela de variação: 2.814814814814815
Existe Diferença Significativa
